In [62]:
# !pip install html2image

# Start Here

In [63]:
import json
import os
import random
import re
import textwrap

from html2image import Html2Image
from nltk.corpus import stopwords, wordnet
from nltk.metrics.distance import edit_distance
import pytrends
import nltk
from tqdm import tqdm

from sumy.nlp.stemmers import Stemmer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer

In [64]:
spell_check_flag = False

In [65]:
stopwords = stopwords.words('english')

In [66]:
with open('ppn_deck.json', 'r') as read_file:
    card_deck = json.load(read_file)



In [67]:
def clear_card_box(card_box_directory):
    for file in os.listdir(card_box_directory):
        os.remove(os.path.join(card_box_directory, file))

# clear the card box directory
clear_card_box('new_card_box')

In [68]:
from PIL import Image

def crop_image(input_file, output_file):
    # Open the input image
    img = Image.open(input_file)

    # Crop the image to maintain a 14px buffer on the left, top, bottom, and right
    # of a rectangle that is 1182px wide and 1779px tall, starting from the upper left
    left = 14 # buffer
    cropped_img = img.crop((left, 14, 1182+left, 1779+left))

    # Save the cropped image to the specified output file
    cropped_img.save(output_file)


In [69]:
with open('card_html.txt', 'r') as f:
    html_template = f.read()

In [70]:
# using contextual spell checking
# verify that python -m spacy download en_core_web_sm has been run
#!python -m spacy download en_core_web_sm

#!pip install textblob
from textblob import TextBlob
#!pip install autocorrect 
from autocorrect import Speller # for spell checking

In [71]:
import spacy
import re
import textblob

import contextualSpellCheck
nlp = spacy.load('en_core_web_sm')
contextualSpellCheck.add_to_pipe(nlp)

In [84]:
def summarize_text(text, num_sentences):
    """
    Summarize the given text using the LSA or LexRank summarization algorithms and return the summary as a string
    """
    # create a PlaintextParser object to parse the text
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    # choose a summarization algorithm
    # algorithm = LsaSummarizer()
    algorithm = LexRankSummarizer()

    # summarize the text and return the summary as a string
    summary = algorithm(parser.document, num_sentences)
    summary_text = "\n".join([str(sentence) for sentence in summary])

    return summary_text

def generate_card(title: str, description: str, points: str, html_template,  name, links_on_wikipedia):
    filename = '{}.png'.format(title.replace(' ', '_'))
    if os.path.exists('./card_box/{}'.format(filename)) and 'Test_Card' not in filename:
        print(f'Card {title} already exists, skipping...')
        return # skip this card if it already exists
    # Set the dimensions in centimeters
    width_cm = 5.5
    height_cm = 8.5
    card_color = 'green'
    # Convert the dimensions to pixels
    # 1 cm is equal to 96 pixels
    width_pixels = int(width_cm * 96)
    height_pixels = int(height_cm * 96)
    points = int(points) # removes the decimal point
    points = str(points) # now it's a string
    links_on_wikipedia = str(links_on_wikipedia)
    # replace the placeholders with the actual values
    html = html_template
    # identify any bigrams from the title that are in the description
    # if there are any, remove them from the description
    # this is to prevent the same information from being revealed in the description from the title
    title_bigrams = list(nltk.bigrams(title.split()))
    # predict the category of the card
    title_features = {} # initialize the title features dictionary to be empty
    for word in title.split():
        title_features['contains({})'.format(word.lower())] = True # add the word to the title features dictionary
    # predict the category of the card
    #!category = max(nltk.classify.util.apply_features(title_features, [title]), key=lambda x: x[1]).label()
    # category = nltk.classify.util.apply_features(title_features, [title])# -> this is a list of tuples, where the first element is the category and the second element is the probability
    # print(category)
    # category = category[0][0] # get the first element of the first tuple in the list, which is the category
    category = 'Wild Card'
 

    # replace the category in the description with 'THIS'




    for bigram in title_bigrams:
        bigram = ' '.join(bigram)
        if bigram in description:
            description = description.replace(bigram, ' THIS {} '.format(category))
    # also, in the description, remove the ' - ' phrase and replace it with 'SUBJECT'
    description = description.replace(' - ', ' ')
    # keep removing double spaces until there are no more double spaces in the description.
    def format_description(description):
        while '  ' in description:
            description = description.replace('  ', ' ')

    description = clean_string(description) # clean the description
    html = html.replace('CARD_TITLE', title)
    html = html.replace('CARD_DESCRIPTION', description)
    html = html.replace('CARD_POINTS', str(points))
    html = html.replace('CARD_NAME',str(title))
    html = html.replace('CARD_COLOR',str(card_color))
    # replace the width_pixels and height_pixels with the actual values
    html = html.replace('WIDTH_PIXELS',str(width_pixels))
    html = html.replace('HEIGHT_PIXELS',str(height_pixels))
    html = html.replace('WIKILINKS',str(links_on_wikipedia))
    # Create an Html2Image object
    hti = Html2Image()

    # Generate a screenshot of the HTML content and save it to a file
    output_path = './card_box/'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    savepath = os.path.join(output_path, filename)
    hti.output_path = output_path


    # Generate the image
    # make the filename the title of the card, with underscores instead of spaces, and no punctuation or special characters.
    filename = '{}.png'.format(title.replace(' ', '_')) # make the filename the title of the card, with underscores instead of spaces, and no punctuation or special characters.
    filename = filename.lower()
    hti.screenshot(html_str=html, save_as=filename)
    # Crop the image (at the filename (path))
    print(f'Generated card {title} at {savepath}!')
    
    savepath_out = './new_card_box/{}.png'.format(title.replace(' ', '_').replace('.', '').replace(',', '').replace(';', '').replace(':', '').replace('?', '').replace('!', '').replace('-', '').replace('"', '').replace("'", '').replace('(', '').replace(')', '')).lower() # make the filename the title of the card, with underscores instead of spaces, and no punctuation or special characters.
    crop_image(savepath, savepath_out)
    # delete the original image
    os.remove(savepath)
    print(f'Cropped card {title} at {savepath}!')

def run_spell_check_on_cards(nlp):
    #note: this function takes a long time to run, very top heavy
    # open the json file
    with open('ppn_deck.json', 'r') as f:
        card_deck = json.load(f)
    
    # loop through the cards
    for card in tqdm(card_deck):
        # get the summary
        summary = card['summary_short']
        if isinstance(summary, list):
            summary = summary[1]

        # using contextualSpellCheck to correct the spelling
        doc = nlp(summary)
        if doc._.performed_spellCheck: # if the spell check was performed
            # reassign summary to the corrected version
            summary = doc._.outcome_spellCheck
            card['summary_short'] = summary # update the card
        else:
            print('Spell check was not performed')

def generate_physical_cards(options):
    # get the options from the options dictionary
    cards_to_generate = options['cards_to_generate'] # the number of cards to generate
    profanity = options['profanity'] # if True, profanity will be allowed in the cards
    grade_level = options['grade_level'] # the maximum grade level of the cards
    keywords = options['keywords'] # if card summary (full) contains any of these keywords, it will be included in the deck of cards as a card with the category matching the keyword. If the keyword is not in the list of categories, it will be added to the list of categories.

    # Iterate through each card and generate a card image for it
    for card in tqdm(card_deck):
        if gandalf_card_finder(card, keywords, grade_level, profanity=False):
            pass # get it?
        else:
            continue
        # Prepare the summary and points variables
        summary = card['summary_short']
        summary = str(summary)
        if isinstance(summary, str):
            # if the summarized text fills more than 3/4 the height of the card, summarize it again with one fewer sentence. This is to prevent the text from overflowing the card.
            # max at font size 20 is 25 lines
            sentence_count = 5 # the number of sentences in the summary (initially)
            while len(summary) > 10 * 40:# and sentence_count > 1:
                summary = summarize_text(summary, sentence_count)
                sentence_count -= 1
            if isinstance(summary, list):
                summary = ' '.join(summary)
        else:
            raise ValueError('Summary is not a string!')
        card['final_summary'] = summary
        points = card['points_for_card']
        # Generate the card
        generate_card(str(card['title']),
                      summary, points=points,
                      html_template=html_template,
                      name=str(card['title']),
                      links_on_wikipedia=card['related'])

def gandalf_card_finder(card, keywords, grade_level, profanity):
    # gandalf scans the card and determines if it is a good card or not for the game. For example, if the card contains any profanity and the profanity is set to false AND it contains a keyword, it is still not a good card because it failed one of the two requirements, with profanity being the most important.
    # if the card is good, it returns True, otherwise it returns False
    assert('summary' in card.keys(), 'Card does not have a summary!')
    full_summary = card['summary']
    if isinstance(full_summary, list):
        full_summary = full_summary[1]
    # check if the card contains any of the keywords using regex and word boundaries (r'\b')
    found_keyword = False # initialize the found_keyword variable
    for keyword in keywords:
        if re.search(r'\b' + keyword + r'\b', full_summary, re.IGNORECASE):
            # if the keyword is found, return True
            found_keyword = True
            break
        else:
            print(f'Keyword {keyword} not found in card {card["title"]}.')
            found_keyword = False
    # profanity check here
    profanity = False #note: this is not implemented yet
    
    #todo - implement profanity check
    #todo - implement grade level check
    grade_level_check = True #note: this is not implemented yet
    # check the grade level of the card
    if float(card['grade_level']) > float(grade_level):
        grade_level_check = False # if the grade level is too high, the card is not a good card
    # if the card is good, return True, otherwise return False
    if (found_keyword and grade_level_check) or grade_level_check:
        return True
    else:
        return False

def clean_string(string):
    # Remove non-alphanumeric characters at the beginning of sentences
    string = re.sub(r'(?<=[.!?])\s*[^a-zA-Z\d]', '', string)
    
    # Use TextBlob to correct syntax errors
    corrected_string = textblob.TextBlob(string).correct()
    
    # convert to string from TextBlob object
    corrected_string = str(corrected_string)
    description = corrected_string
    description = re.sub(r'\.(?=[^ ])', '. ', description)
    description = re.sub(r',(?=[^ ])', ', ', description)
    description = re.sub(r';(?=[^ ])', '; ', description)
    description = re.sub(r':(?=[^ ])', ': ', description)
    description = re.sub(r'\?(?=[^ ])', '? ', description)
    description = re.sub(r'!(?=[^ ])', '! ', description)
    description = re.sub(r'-(?=[^ ])', '- ', description)
    description = re.sub(r'"(?=[^ ])', '" ', description)
    description = re.sub(r"'(?=[^ ])", "' ", description)
    description = re.sub(r'\((?=[^ ])', '( ', description)
    description = re.sub(r'\)(?=[^ ])', ') ', description)
    corrected_string = description
    # Remove non-ASCII characters
    cleaned_string = re.sub(r'[^\x00-\x7F]+', '', corrected_string)

    # remove \n characters from the string
    cleaned_string = cleaned_string.replace('\n', ' ')

    # capitalize the first letter of each sentence in the description
    description = description.capitalize() # capitalize the first letter of the description
    # capitalize the first letter of each sentence in the description
    description = re.sub(r'[\.\?\!]\s(?=[a-z])', '. ', description) # capitalize the first letter of each sentence, except for the first sentence, which is already capitalized, and except for sentences that start with a number, which are not capitalized in the original text
    # replace the placeholders with the actual values
    return cleaned_string

def run_cleaner_on_cards():
    # open the json file
    with open('ppn_deck.json', 'r') as f:
        card_deck = json.load(f)

    # loop through the cards
    for card in tqdm(card_deck):
        # get the summary
        if 'cleaned' in card.keys():
            continue
        summary = card['summary_short']
        if isinstance(summary, list):
            summary = summary[1]
        if isinstance(summary, list):
            summary = ' '.join(summary)
        # convert summary to string in case it's a list
        summary = str(summary)
        # clean the summary
        summary = clean_string(summary)
        # update the card
        card['summary_short'] = summary
        # add a new key to the card called 'cleaned' and set it to True
        card['cleaned'] = True
        # every 10 cards, save the json file
        if card_deck.index(card) % 10 == 0:
            #print(f'Saving at card {card_deck.index(card)}')
            with open('ppn_deck.json', 'w') as f:
                json.dump(card_deck, f)
        #print(f'Cleaned card {card["title"]} at index {card_deck.index(card)}! Summary: {summary}')
    # save the updated json file
    with open('ppn_deck.json', 'w') as f:
        json.dump(card_deck, f)

<>:164: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:164: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/var/folders/34/d1tlq3k91hb0lj6x90xpzb4r0000gn/T/ipykernel_96892/225915091.py:164: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert('summary' in card.keys(), 'Card does not have a summary!')


In [85]:
print("Initialized, ready to spell check!")
# run_spell_check_on_cards(nlp) # run the spell check
if spell_check_flag:
    run_cleaner_on_cards()
print("Spell check complete, ready to generate cards!")


Initialized, ready to spell check!
Spell check complete, ready to generate cards!


In [86]:
# generate a test card
generate_card('Test Card', 'from Armentires is an English song that was particularly popular during World War I. It is also known by its ersatz French hook line, Inky Pinky Parlez Vous, or the American variant Hinky Dinky Parlez-vous variant: Parlay voo. Inky Pinky was a Scottish childrens name for parsnip and potato cakes, but it has been suggested that an onomatopoeic reference to the sound of bed springs is a more likely soldiers ribald derivation.', 10, html_template, 'Mademoiselle from Armentières', 0)

objc[19148]: Class WebSwapCGLLayer is implemented in both /System/Library/Frameworks/WebKit.framework/Versions/A/Frameworks/WebCore.framework/Versions/A/Frameworks/libANGLE-shared.dylib (0x7ffa59435aa0) and /Applications/Google Chrome.app/Contents/Frameworks/Google Chrome Framework.framework/Versions/108.0.5359.124/Libraries/libGLESv2.dylib (0x11442b568). One of the two will be used. Which one is undefined.
[1227/160237.029495:INFO:headless_shell.cc(623)] Written to file /Users/grahamwaters/Library/Mobile Documents/com~apple~CloudDocs/Mimikers/card_box/test_card.png.


Generated card Test Card at ./card_box/Test_Card.png!
Cropped card Test Card at ./card_box/Test_Card.png!


In [90]:
# set the options for the game
options = {
    'grade_level': 12, # maximum grade level of the cards
    'profanity': True,
    'cards_to_generate': 10,
    'keywords': [],
    'categories': [],
}


In [91]:
# keywords that we want to include from the cards in the deck (cards with these words should be included in the deck).
keywords = [
    'amish',
    'funny',
    'nasa',
    'famous',
    'viral',
    'dog poop',
    'poop',
    'therapy',
    'vegetables',
    'melon',
    'die',
    'embarrassing'
]

In [92]:
generate_physical_cards(options)
print("Done")

  0%|          | 0/9296 [00:00<?, ?it/s]objc[20158]: Class WebSwapCGLLayer is implemented in both /System/Library/Frameworks/WebKit.framework/Versions/A/Frameworks/WebCore.framework/Versions/A/Frameworks/libANGLE-shared.dylib (0x7ffa59435aa0) and /Applications/Google Chrome.app/Contents/Frameworks/Google Chrome Framework.framework/Versions/108.0.5359.124/Libraries/libGLESv2.dylib (0x114c01568). One of the two will be used. Which one is undefined.
[1227/160356.991205:INFO:headless_shell.cc(623)] Written to file /Users/grahamwaters/Library/Mobile Documents/com~apple~CloudDocs/Mimikers/card_box/rosetta_cammeniti.png.


Generated card Rosetta Cammeniti at ./card_box/Rosetta_Cammeniti.png!


  0%|          | 2/9296 [00:06<7:52:59,  3.05s/it]

Cropped card Rosetta Cammeniti at ./card_box/Rosetta_Cammeniti.png!


objc[20210]: Class WebSwapCGLLayer is implemented in both /System/Library/Frameworks/WebKit.framework/Versions/A/Frameworks/WebCore.framework/Versions/A/Frameworks/libANGLE-shared.dylib (0x7ffa59435aa0) and /Applications/Google Chrome.app/Contents/Frameworks/Google Chrome Framework.framework/Versions/108.0.5359.124/Libraries/libGLESv2.dylib (0x112401568). One of the two will be used. Which one is undefined.
[1227/160358.964989:ERROR:command_buffer_proxy_impl.cc(128)] ContextResult::kTransientFailure: Failed to send GpuControl.CreateCommandBuffer.
[1227/160359.809918:INFO:headless_shell.cc(623)] Written to file /Users/grahamwaters/Library/Mobile Documents/com~apple~CloudDocs/Mimikers/card_box/spy_muppets:_license_to_croak.png.


...

Generated card Fame in the 20th Century at ./card_box/Fame_in_the_20th_Century.png!


  0%|          | 4/9296 [00:11<6:56:05,  2.69s/it]

Cropped card Fame in the 20th Century at ./card_box/Fame_in_the_20th_Century.png!


objc[20295]: Class WebSwapCGLLayer is implemented in both /System/Library/Frameworks/WebKit.framework/Versions/A/Frameworks/WebCore.framework/Versions/A/Frameworks/libANGLE-shared.dylib (0x7ffa59435aa0) and /Applications/Google Chrome.app/Contents/Frameworks/Google Chrome Framework.framework/Versions/108.0.5359.124/Libraries/libGLESv2.dylib (0x1146bf568). One of the two will be used. Which one is undefined.
[1227/160403.593434:ERROR:command_buffer_proxy_impl.cc(128)] ContextResult::kTransientFailure: Failed to send GpuControl.CreateCommandBuffer.
[1227/160404.495805:INFO:headless_shell.cc(623)] Written to file /Users/grahamwaters/Library/Mobile Documents/com~apple~CloudDocs/Mimikers/card_box/becca_hayton.png.


Generated card Becca Hayton at ./card_box/Becca_Hayton.png!
